In [1]:
import requests # 웹 페이지 소스를 얻기 위한 패키지(기본 내장 패키지이다.)
from bs4 import BeautifulSoup # 웹 페이지 소스를 얻기 위한 패키지, 더 간단히 얻을 수 있다는 장점이 있다고 한다.
from datetime import datetime                                # (!pip install beautifulsoup4 으로 다운받을 수 있다.)
import pandas as pd # 데이터를 처리하기 위한 가장 기본적인 패키지
import time # 사이트를 불러올 때, 작업 지연시간을 지정해주기 위한 패키지이다. (사이트가 늦게 켜지면 에러가 발생하기 때문)
import urllib.request #
from selenium import webdriver
import json
import re     
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt
import numpy as np
import pickle
import matplotlib

import matplotlib.pyplot as plt

import numpy as np

import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
from pandas.io.json import json_normalize
import seaborn as sns
import statsmodels.api as sm
import requests

In [2]:
challenger_timeline10 =pd.read_csv('/Users/sinmin-yong/GitHub/studies/Riot_DB/Challenger_Ranked_Games_10minute.csv')

In [3]:
challenger_timeline10.head()

,gameId,blueWins,blueTotalGolds,blueCurrentGolds,blueTotalLevel,blueAvgLevel,blueTotalMinionKills,blueTotalJungleMinionKills,blueFirstBlood,blueKill,...,redFirstTowerLane,redTowerKills,redMidTowerKills,redTopTowerKills,redBotTowerKills,redInhibitor,redFirstDragon,redDragnoType,redDragon,redRiftHeralds
0,4247263043,0,14870,2889,32,6.4,199,53,0,3,...,[],0,0,0,0,0,1,['WATER_DRAGON'],1,0
1,4247155821,1,14497,2617,33,6.6,229,44,0,2,...,[],0,0,0,0,0,0,[],0,0
2,4243963257,0,15617,1757,34,6.8,223,39,0,3,...,['BOT_LANE'],1,0,0,1,0,1,['FIRE_DRAGON'],1,1
3,4241678498,0,15684,1439,35,7.0,251,64,0,3,...,[],0,0,0,0,0,0,[],0,0
4,4241538868,1,17472,3512,35,7.0,257,46,0,7,...,[],0,0,0,0,0,0,[],0,0


In [5]:
challenger_timeline10.dtypes

gameId                          int64
blueWins                        int64
blueTotalGolds                  int64
blueCurrentGolds                int64
blueTotalLevel                  int64
blueAvgLevel                  float64
blueTotalMinionKills            int64
blueTotalJungleMinionKills      int64
blueFirstBlood                  int64
blueKill                        int64
blueDeath                       int64
blueAssist                      int64
blueWardPlaced                  int64
blueWardKills                   int64
blueFirstTower                  int64
blueFirstInhibitor              int64
blueFirstTowerLane             object
blueTowerKills                  int64
blueMidTowerKills               int64
blueTopTowerKills               int64
blueBotTowerKills               int64
blueInhibitor                   int64
blueFirstDragon                 int64
blueDragnoType                 object
blueDragon                      int64
blueRiftHeralds                 int64
redWins     

In [6]:
# 데이터를 이렇게 만드니까 분석하기가 너무 까다롭군...
# 분리작업을 거쳐야겠네
blue_ls = []
red_ls = []
for i in challenger_timeline10.columns.tolist():
    if i == 'gameId':
        continue
    elif 'blue' in i:
        blue_ls.append(i)
        
    elif 'red' in i:
        red_ls.append(i)

In [10]:
blue_df = challenger_timeline10[blue_ls]
red_df = challenger_timeline10[red_ls]

col_ls = []
for i in blue_df.columns.tolist():
    col_ls.append(re.sub('blue','',i))

In [12]:
blue_df.columns = col_ls
red_df.columns = col_ls

In [13]:
ch_df = pd.concat([blue_df,red_df],axis=0)

In [14]:
ch_df

,Wins,TotalGolds,CurrentGolds,TotalLevel,AvgLevel,TotalMinionKills,TotalJungleMinionKills,FirstBlood,Kill,Death,...,FirstTowerLane,TowerKills,MidTowerKills,TopTowerKills,BotTowerKills,Inhibitor,FirstDragon,DragnoType,Dragon,RiftHeralds
0,0,14870,2889,32,6.4,199,53,0,3,9,...,[],0,0,0,0,0,0,[],0,0
1,1,14497,2617,33,6.6,229,44,0,2,3,...,[],0,0,0,0,0,0,[],0,1
2,0,15617,1757,34,6.8,223,39,0,3,11,...,[],0,0,0,0,0,0,[],0,0
3,0,15684,1439,35,7.0,251,64,0,3,4,...,[],0,0,0,0,0,0,[],0,0
4,1,17472,3512,35,7.0,257,46,0,7,5,...,[],0,0,0,0,0,0,[],0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26404,1,14779,3709,36,7.2,205,67,0,5,7,...,[],0,0,0,0,0,1,['WATER_DRAGON'],1,1
26405,1,42001,9568,44,8.8,279,73,0,27,18,...,[],3,1,0,2,0,0,[],0,0
26406,1,14845,1345,34,6.8,239,60,0,0,3,...,[],0,0,0,0,0,0,[],0,0
26407,0,14773,3673,33,6.6,204,52,0,4,6,...,[],0,0,0,0,0,0,[],0,0
